In [155]:
import numpy as np

class AntColony(object):

    def __init__(self, num_cities, num_ants, num_best):
        """
        Args:
            num_best (int): Number of best ants who deposit pheromone
        """
        self.distances_matrix = self.generate_distances(num_cities)
        self.pheromone_matrix = np.array([0.01]*num_cities*num_cities).reshape(num_cities, num_cities)
        self.cities = np.arange(num_cities)
        self.num_ants = num_ants
        self.num_best = num_best
        #fixed values
        self.num_iterations = 50
        self.evaporation_rate = 0.8
        self.alpha = 0.1
        self.beta = 1.0
        print(self.distances_matrix)
        
    def generate_distances(self, num_cities):
        distance = np.zeros((num_cities, num_cities))
        for i in range(num_cities):
            for j in range(i+1, num_cities):
                distance[i][j] = distance[j][i] = np.random.randint(4,50)
        return distance 
        
    def run(self):
        shortest_path = None
        best_path_so_far = ("", np.inf)
        for i in range(self.num_iterations):
            all_paths = self.generate_all_paths()
            self.spread_pheromone(all_paths)
            shortest_path = min(all_paths, key=lambda x: x[1])
            if shortest_path[1] < best_path_so_far[1]:
                best_path_so_far = shortest_path            
            self.pheromone_matrix *= self.evaporation_rate            
        return best_path_so_far

    def spread_pheromone(self, all_paths):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:self.num_best]:
            for move in path:
                self.pheromone_matrix[move[0], move[1]] += 1.0 / self.distances_matrix[move[0], move[1]]

    def calculate_path_distance(self, path):
        total_distance = 0
        for ele in path:
            total_distance += self.distances_matrix[ele[0], ele[1]]
        return total_distance

    def generate_all_paths(self):
        all_paths = []
        for i in range(self.num_ants):
            path = self.generate_path(0)
            all_paths.append((path, self.calculate_path_distance(path)))
        return all_paths

    def generate_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for i in range(len(self.distances_matrix) - 1):
            move = self.select_next_move(self.pheromone_matrix[prev], self.distances_matrix[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start)) # going back to where we started    
        return path

    def select_next_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0

        indices = np.where(pheromone != 0)[0]
        row = (pheromone[indices] ** self.alpha) * ((1.0 / dist[indices]) ** self.beta)

        norm_row = row / row.sum()
        next_move = np.random.choice(indices, 1, p=norm_row)[0]
        return next_move
    

# Example usage:
ant_colony = AntColony(4, 20, 5)
shortest_path = ant_colony.run()
print("shortest_path:", shortest_path)


[[ 0. 18. 18. 31.]
 [18.  0. 22. 42.]
 [18. 22.  0. 43.]
 [31. 42. 43.  0.]]
shortest_path: ([(0, 2), (2, 1), (1, 3), (3, 0)], 113.0)
